In [ ]:
### 🔧 0. 환경설정  (필요한 경로만 수정하세요) ############################
DATA_ROOT  = "data/processed/zarr/train"       # Zarr 폴더
TRAIN_CSV  = "data/raw/train_labels.csv"       # fold 정보 포함 CSV
FOLD_ID    = 0                                 # 확인할 fold
DEVICE     = "cuda"                            # 또는 "cpu"

# 아주 작은 cfg – common_config · argparse 대신 하드코딩
CFG_DICT = dict(
    seed=777,
    roi_size=(96,96,96),
    train_sub_epochs=1,
    num_workers=4,
    prefetch_factor=2,
    batch_size=4,
    train_sampler="oversample",
    pos_ratio = .8,
)

#######################################################################


In [ ]:
# ───────────────────────────────────────────────────────
# notebook_cell [0]  : 기본 경로 & PYTHONPATH
# ───────────────────────────────────────────────────────
import sys, os
from pathlib import Path

# 프로젝트 루트
PRJ = Path(r"D:\project\Kaggle\BYU\byu-motor").resolve()

# src 모듈 import
for p in [PRJ, PRJ / "src"]:
    if str(p) not in sys.path:
        sys.path.insert(0, str(p))

print("sys.path[0:2] =", sys.path[:2])

In [ ]:
# notebook_cell [1]
import copy
from src.configs.common_config import base_cfg   # 우리가 만든 common_config
from src.configs.cfg_motor_resnet34 import cfg as res34_cfg
from src.configs.cfg_motor_effb3   import cfg as effb3_cfg   # 필요 시

# ────────── CSV 경로 교체 ──────────
CSV_FOLDED = PRJ / "data" / "raw" / "train_labels_folds.csv"

# 학습 시 사용할 공통 옵션
COMMON = dict(
    train_df=str(CSV_FOLDED),
    val_df=None,                # fold 컬럼으로 자동 분리
    data_folder=str(PRJ / "data"),         # raw/processed/zarr 하위
    output_dir=str(PRJ / "outputs"),
    num_workers=8,
    pin_memory=True,
)

print("Fold CSV  :", CSV_FOLDED)


In [ ]:
### 🔍 1. DataLoader 구동 & key 확인 ####################################
from types import SimpleNamespace
from pathlib import Path
import pandas as pd, time, torch, inspect, importlib, sys

# ---- 1-a)  모듈 로드 (로컬 src 경로가 PYTHONPATH에 없다면 추가)
PROJECT_ROOT = Path().resolve()        # 필요하면 수정
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from datasets.motor_dataset import MotorDataset, tr_collate_fn, val_collate_fn
from datasets.sampler import PosBgSampler
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

cfg = SimpleNamespace(**CFG_DICT)

# ---- 1-b)  CSV → fold 분리
df = pd.read_csv(TRAIN_CSV)
val_df = df[df["fold"] == FOLD_ID].copy()

ds_val = MotorDataset(val_df, cfg,
                      data_root=DATA_ROOT,
                      mode="val",
                      stride=1,          # 당장 stride 안 쓴다 해도 1로
                      aug=None)

sampler   = SequentialSampler(ds_val)
val_loader= DataLoader(ds_val,
                       batch_size=cfg.batch_size,
                       sampler=sampler,
                       num_workers=cfg.num_workers,
                       pin_memory=True,
                       collate_fn=val_collate_fn,
                       prefetch_factor=cfg.prefetch_factor,
                       persistent_workers=True)

print(f"✅  val dataset len = {len(ds_val):,}")


In [ ]:
### 🕒 2. 배치 하나 꺼내서 key / shape / 소요시간 체크 ###################
t0 = time.time()
batch = next(iter(val_loader))
print(f"⏱️  first val batch loaded in {time.time()-t0:.2f}s")

for k,v in batch.items():
    if torch.is_tensor(v):
        print(f"{k:10s}  tensor  {tuple(v.shape)}  dtype={v.dtype}")
    else:
        # e.g. list[str] …
        print(f"{k:10s}  {type(v).__name__}  len={len(v)}")

assert "roi_start" in batch, "`roi_start` key가 없음 ‼️"
assert batch["image"].shape[2:]==cfg.roi_size, "patch size 불일치"


In [ ]:
### ⚡ 3. 1 epoch · 평가 루프가 오래 걸리는 이유 탐색용 ###################
#   배치마다 시간 측정해서 어디서 끊기는지 확인
def timed_val_pass(loader):
    times = defaultdict(list)
    for i,batch in enumerate(loader):
        t = time.time()
        _ = batch["image"].to(DEVICE, non_blocking=True)
        times["to(cuda)"].append(time.time()-t)

        # ─ 여기에 모델 forward 를 넣어도 되고, 일단은 스킵 ─
        if i==10: break   # 11 step 으로 충분
    return {k: sum(v)/len(v) for k,v in times.items()}

print("실행 중 …")
print(timed_val_pass(val_loader))
